# Wpowadzenie do deep learning w bibliotece Flux.jl

## Przykład

Aby w  zrozumieć sposób pracy z Fluxem warto rozpatrzeć prosty przykład. Zajmiemy się przetwarzaniem języka naturalnego - zbudujemy model zdolny do generowania składnej wypowiedzi w języku polskim.

Wyjściowe założenie jest takie, że wytrenujemy sieć neuronową, która będzie estymowała prawdopodobieństwo wystąpienia danego znaku w ciągu na podstawie poprzedzających go znaków w sekwencji ([<b>Character-Level Language Model</b>](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)).

Zbiórem na którym będziemy pracowali jest <i>W poszukiwaniu straconego czasu</i> Marcela Prousta. 

[![](https://upload.wikimedia.org/wikipedia/commons/b/b8/Marcel_Proust_1895.jpg)](https://pl.wikipedia.org/wiki/Marcel_Proust)

>(...) matka widząc, że mi jest zimno, namówiła mnie, abym się napił wbrew zwyczajowi trochę herbaty. Odmówiłem zrazu; potem, nie wiem czemu, namyśliłem się. Posłała po owe krótkie i pulchne ciasteczka zwane magdalenkami, które wyglądają jak odlane w prążkowanej skorupie muszli. I niebawem (...) machinalnie podniosłem do ust łyżeczkę herbaty, w której rozmoczyłem kawałek magdalenki. Ale w tej samej chwili, kiedy łyk pomieszany z okruchami ciasta dotknął mego podniebienia, zadrżałem, czując, że się we mnie dzieje coś niezwykłego. Owładnęła mną rozkoszna słodycz (...). Sprawiła, że w jednej chwili koleje życia stały mi się obojętne, klęski jako błahe, krótkość złudna (...). Cofam się myślą do chwili, w której wypiłem pierwszą łyżeczkę herbaty (...). I nagle wspomnienie zjawiło mi się. Ten smak to była magdalenka cioci Leonii.(...)

Zanim jednak zaczniemy wprowadźmy odrobinę teorii stojącej za tym zagadnieniem:

### Rekurencyjne sieci neuronowe (Recurrent neural networks)

- Charakterystyczną cechą tego typu sieci jest to, że pozwalają one na istnienie wewnątrz grafu cykli skierowanych.
- Oznacza to, że informacja wewnątrz takiej sieci nie musi płynąć tylko w jednym kierunku - neurony leżące na tej samej warstwie także mogą przesyłać sobie wzajemnie dane:

[![](http://karpathy.github.io/assets/rnn/diags.jpeg)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

Dzięki tej właściwości RNN doskonale nadają się do budowy interesującego nas modelu: 

[![](http://karpathy.github.io/assets/rnn/charseq.jpeg)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

### Long short-term memory

Problemem na który można natrafić w przypadku korzystania z RNN jest pamięć takiej sieci. Gdy odległość pomiędzy aktualnym a poprzedzającymi go węzłami, które niosą za sobą kluczową informację jest niewielka, sieć jest w stanie efektywnie je wykorzystać:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-shorttermdepdencies.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Problem się pojawia gdy ta odległość jest duża - wtedy kluczowe informacje po prostu znikają w szumie:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-longtermdependencies.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Wtedy też, warto zastosować sieć LSTM, która ze względu na swoją architekturę jest w stanie odpowiednio filtrować informację i wykorzystawać je nawet wtedy, gdy ich źródło jest znacznie oddalone od aktualnego neuronu:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

### Alternatywy

Zamiast bazowych sieci rekurencyjnych lub sieci LSTM (i ich [modyfikacji](https://en.wikipedia.org/wiki/Long_short-term_memory)) można zastosować różne alternatywy, np. sieci <b>Gated Recurrent Unit<b> (GRU):
    
[![](https://upload.wikimedia.org/wikipedia/commons/5/5f/Gated_Recurrent_Unit.svg)](https://en.wikipedia.org/wiki/Gated_recurrent_unit)

Lub inne modele skonstruowane do rozwiązywania specyficznych problemów, np. [uczenia na szeregach czasowych.](https://github.com/sdobber/FluxArchitectures)

Przejdźmy teraz do implementowania modelu za pomocą Fluxa:

### Implementacja

In [1]:
using Flux
using Flux: onehot, argmax, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition
using BSON

Pierwszym krokiem jest oczywiście odpowiednie przygotowanie danych na których będziemy pracowali:

In [2]:
text = collect(read("w_poszukiwaniu.txt",String));
alphabet = [unique(text)..., '_'];

Następnie kodujemy zmienne jakościowe:

In [3]:
text = map(ch -> onehot(ch, alphabet), text);
stop = onehot('_', alphabet);

In [4]:
N = length(alphabet);
seqlen = 100;
batch_size = 32;

In [5]:
Xs = collect(partition(batchseq(chunk(text, batch_size), stop), seqlen));
Ys = collect(partition(batchseq(chunk(text[2:end], batch_size), stop), seqlen));

In [6]:
m = Chain(
    LSTM(N, 128),
    Dropout(0.3), 
    LSTM(128, 128),
    Dropout(0.3),
    Dense(128, N),
    softmax)

function loss(xs, ys)
  l = sum(crossentropy.(m.(xs), ys))
  #Flux.reset!(m)
  return l
end

opt = ADAM(0.001)


function sample(m, alphabet, len; temp = 1)
  Flux.reset!(m)
  buf = IOBuffer()
  c = rand(alphabet)
  for i = 1:len
    write(buf, c)
    c = wsample(alphabet, m(onehot(c, alphabet)))
  end
  return String(take!(buf))
end

#evalcb = () -> @show loss(Xs[5], Ys[5])

evalcb = function ()
    @show loss(Xs[5], Ys[5])
    println(sample(m, alphabet, 100))
end

#4 (generic function with 1 method)

In [7]:
sample(m, alphabet, 50)

"5K»wr5ä0ĆGëP/ZSïżà*xÀG bBZ'ï(J*—:AoÉfu/Xô:HnĘzHĘés"

In [8]:
@info("Beginning training loop...")
best_ls = Inf
last_improvement = 0
for epoch = 1:20
    @info "Epoch: $epoch"
    global best_ls, last_improvement
    Flux.train!(loss, params(m), zip(Xs, Ys), opt, cb=throttle(evalcb, 240))
    ls = loss(Xs[5], Ys[5])
    if ls <= best_ls
        @info "New best result: $ls"
        BSON.@save "char_model.bson" m
        best_ls = ls
        last_improvement = epoch
    end
    if epoch - last_improvement >= 3
        @warn(" -> We're calling this converged.")
        break
    end
end

┌ Info: Beginning training loop...
└ @ Main In[8]:1


loss(Xs[5], Ys[5]) = 480.30615f0
żĆ”w*hÓM6OxrWçU”

┌ Info: Epoch: 1
└ @ Main In[8]:5



loss(Xs[5], Ys[5]) = 313.31958f0
Q7S;?śmĆöpoąyno wwłz aor  i,mmkdakcnc ziebSo  ne jłjłzoę.y nym o— .reiowbcreeł, i urusąw.enca,ż  ,ił
loss(Xs[5], Ys[5]) = 298.99417f0
o7Óámujdoz kną ,   wiewa tamę cbysztoLgsdeieciaco tak ZyjpsoameOnChmózciypead prcoks  oai lojon„w,wg
loss(Xs[5], Ys[5]) = 285.67868f0
tO½Kfôaęyąacy rorbokadamiś waśdogoprzy j mirszoj ła rhdtłiznawaw w łwoć sze  za ziectaynteirzosolimn
loss(Xs[5], Ys[5]) = 273.50592f0
Ź:źsyyrazio płć mą łarszebezyacorbuw ępikrwłp naowczagoobnceśele ru ciezy jagadosbą Ksć., : dtkcneła
loss(Xs[5], Ys[5]) = 267.60925f0
_HkfÉtetrodypozośwn bledmodpo bi wicrwanszosię  gom zytt go,stodeic ,łałemn udyć zo etrnią j sretczt
loss(Xs[5], Ys[5]) = 265.0499f0
gêguĘ gn o jyga chcęcugsimk ny dow wię jeą ilo ueitasceiarÓwtoly—naryn ła aca  zósżesm dwą a nobi, h
loss(Xs[5], Ys[5]) = 259.6283f0
Ę,6xdredmak adaznak ono mymuneiale,lyci mąga il mierzantęk  ś, a„fie rodpom zyłolkwigsupnijneś i. b 


┌ Info: New best result: 259.65012
└ @ Main In[8]:10
┌ Info: Epoch: 2
└ @ Main In[8]:5


...

┌ Info: Epoch: 17
└ @ Main In[8]:5


loss(Xs[5], Ys[5]) = 224.1458f0
ćJbidcpo, strzystniem, proocheskist riośnejundach oście byłercz z iwziedynięktowadabych zadatajwiepo
loss(Xs[5], Ys[5]) = 224.12708f0
àtIprrzy kosąnią, aby zamwawouniej ponbej w z, i któraćc patenenczaslego dla gdyś — sloktennie zds, 
loss(Xs[5], Ys[5]) = 224.39607f0
biAież w kraj jesccemnie bosiać Owtólrón jęcałelnia a, ży tawianiądamau otenrzestbudta nieiegru modn
loss(Xs[5], Ys[5]) = 224.63016f0
,2rbedsru i się da rowzytam doduzi,m wiejwiał nisktuwowiem, w źrzyn,madolrte pososr wielpo nasażurji
loss(Xs[5], Ys[5]) = 224.62314f0
hJcSwnię mialalyłemich udzstorzem, anegodotił , któw, a co szes Brzywyj dla zlilałło chodk wysei nie
loss(Xs[5], Ys[5]) = 224.49005f0
cóąopoże mięoś sy przyść porzyspudnię się judekugpać, ichcoją oborzety, dzyśliszkdadziedzy zlebrerow
loss(Xs[5], Ys[5]) = 224.8943f0
їioóowawiąprowjąże rów zrowacatesa była aniećś, już przę tem tego przymnn, dzi (dajać swosnych rszys
loss(Xs[5], Ys[5]) = 224.49176f0
uhnodtlgota (wnicich panie; kus

┌ Info: Epoch: 18
└ @ Main In[8]:5



loss(Xs[5], Ys[5]) = 223.87787f0
Guuraczyję poszą wy wego ę dowywa? — Dódłaczak  tią Agodawny wiepwej Przemo się zarmnlaj pone o mie 
loss(Xs[5], Ys[5]) = 224.55725f0
pAmezaprnośkapie w alicz, blaby, że się j szy bysią. Nus z s S.

— Pakiostarzywiełpaliszacieniegło
loss(Xs[5], Ys[5]) = 224.5799f0
7pbaa z dorujęcie jarlzrzeują prgo.
 tą, znichś chwczą przejak mo które erokogieła onstegośnościają
loss(Xs[5], Ys[5]) = 224.81987f0
2zGiejak kuętc Eodukielnie lirtdopega, krzeupreażożpoguś ckętnrjowią, ność słi wobę kie —  i żęce ki
loss(Xs[5], Ys[5]) = 224.51291f0
:Nniyszytateszęmą wą cholognięczewi która chwie prośżęwynodł się — nie niać móo po, jej wych, którza
loss(Xs[5], Ys[5]) = 225.32204f0
hhzDilrororeci  chanie Ho mniej ć podla do jeknojedzo  z paługowire przazm niejasjoj stme un janiear
loss(Xs[5], Ys[5]) = 225.57271f0
tszyona i  dlo ywa tarala ma, te, ołuzie tarochurziej nie pode zadzuszyjącą tustad znienoddziątywy ł


┌ Warning:  -> We're calling this converged.
└ @ Main In[8]:16


In [9]:
BSON.@load "char_model.bson" m

In [10]:
sample(m, alphabet, 50)

"YSąodię jaka darrzezm, biem się nie, przejakzał w "

## Dodatkowa praca domowa

Zmodyfikuj kod tak, aby poprawić jakość generowanego tekstu <b>(5 punktów)</b>.